# Data discretization

In [1]:
# modules used
import numpy as np
import pandas as pd
import pickle
import os
import re
import mmh3
import operator
import matplotlib.pyplot as plt

In [4]:
# read data if not existing yet
# THIS TAKES A DECENT AMOUNT OF TIME
# USE THE .pkl FILE
if os.path.exists("task3.pkl"):
    data = pd.read_pickle("task3.pkl")
    
else:
    file = open('capture20110818.pcap.netflow.labeled', 'r')
    header = file.readline()
    header = ' '.join(header.split()).split(' ')
    rows_list = []
#     data = pd.DataFrame(columns=["Date", "Start", "Dur", "Prot", "Src", "PortSrc", "Dest", 
#                                  "PortDest", "Flags", "Tos", "Packets", "Bytes", "Flows", "Label"])
    for line in file:
        dict = {}
        words = ' '.join(line.split()).split(' ')
        if len(words) >= 13:
            if words[12] == "Background":
                continue
        elif words[11] == "Background":
                continue
        # split ip adress on adress and port
        dest = words[6].split(':')
        src = words[4].split(':')
        dict["Date"] = words[0]
        dict["Start"] = words[1]
        dict["Dur"] = words[2]
        dict["Prot"] = words[3]
        dict["Src"] = src[0]
        if len(src) > 1:
            dict["PortSrc"] = src[1]
        else:
            dict["PortSrc"] = None
        dict["Dest"] = dest[0]
        if len(dest) > 1:
            dict["PortDest"] = dest[1]
        else:
            dict["PortDest"] = None
        if len(words) == 13:
            dict["Flags"] = words[7]
            dict["Tos"] = words[8]
            dict["Packets"] = words[9]
            dict["Bytes"] = words[10]
            dict["Flows"] = words[11]
            dict["Label"] = words[12]
        elif words[7] == "S_" or words[7] == "SRPA_":
            dict["Flags"] = words[7]
            dict["Tos"] = words[8]
            dict["Packets"] = words[9]
            dict["Bytes"] = words[10]
            dict["Flows"] = words[11]
            dict["Label"] = words[12]
            
        else:
            dict["Flags"] = None
            dict["Tos"] = words[7]
            dict["Packets"] = words[8]
            dict["Bytes"] = words[9]
            dict["Flows"] = words[10]
            dict["Label"] = words[11]
        rows_list.append(dict)
    data = pd.DataFrame(rows_list, columns=["Date", "Start", "Dur", "Prot", "Src", "PortSrc", "Dest", 
                                 "PortDest", "Flags", "Tos", "Packets", "Bytes", "Flows", "Label"])
    data.to_pickle("task3.pkl")

data.head(100)

,Date,Start,Dur,Prot,Src,PortSrc,Dest,PortDest,Flags,Tos,Packets,Bytes,Flows,Label
0,2011-08-18,10:19:13.347,4.985,TCP,147.32.80.13,80,147.32.85.88,56949,PA_,0,91,86277,1,LEGITIMATE
1,2011-08-18,10:19:13.392,0.000,TCP,147.32.86.110,48102,74.125.232.214,443,A_,0,1,66,1,LEGITIMATE
2,2011-08-18,10:19:13.411,4.921,TCP,147.32.85.88,56949,147.32.80.13,80,A_,0,49,3234,1,LEGITIMATE
3,2011-08-18,10:19:13.460,4.742,TCP,147.32.84.59,2768,74.125.108.208,80,A_,0,118,7080,1,LEGITIMATE
4,2011-08-18,10:19:13.486,0.000,TCP,147.32.84.59,56058,74.125.232.215,443,A_,0,1,60,1,LEGITIMATE
5,2011-08-18,10:19:13.689,4.434,TCP,147.32.84.164,34588,195.24.232.164,80,A_,0,30,1980,1,LEGITIMATE
6,2011-08-18,10:19:13.848,0.000,TCP,147.32.86.110,53538,74.125.232.215,443,A_,0,1,66,1,LEGITIMATE
7,2011-08-18,10:19:13.858,0.174,TCP,147.32.84.59,37578,74.125.232.215,443,PA_,0,5,559,1,LEGITIMATE
8,2011-08-18,10:19:14.067,0.009,TCP,147.32.85.26,58763,209.85.148.147,80,FA_,0,2,120,1,LEGITIMATE
9,2011-08-18,10:19:14.134,0.000,TCP,147.32.84.21,44146,74.125.232.216,443,A_,0,1,66,1,LEGITIMATE


# Data exploration
We check one infected host to pick two features that are most relevant for classification

In [16]:
# most occuring infected host is 147.32.84.191
print data.loc[(data["Label"] == "Botnet") & (data["Src"].isin(["147.32.84.191"]) | data["Dest"].isin(["147.32.84.191"]))]

147.32.96.69     100394
147.32.84.59      90560
147.32.80.13      36949
147.32.84.205     24572
147.32.84.206     23169
147.32.84.208     22844
147.32.84.204     22804
147.32.84.207     21861
147.32.84.191     21494
147.32.84.192     21473
Name: Src, dtype: int64
              Date         Start    Dur  Prot             Src PortSrc  \
23985   2011-08-18  10:43:13.001  0.000   UDP   147.32.84.191    1025   
23986   2011-08-18  10:43:13.002  0.000   UDP     147.32.80.9      53   
24032   2011-08-18  10:43:15.259  0.000   UDP   147.32.84.191    1025   
24033   2011-08-18  10:43:15.269  0.000   UDP     147.32.80.9      53   
24034   2011-08-18  10:43:15.270  0.038   TCP   147.32.84.191    1027   
24035   2011-08-18  10:43:15.278  0.027   TCP  74.125.232.201      80   
24067   2011-08-18  10:43:18.170  0.000   UDP   147.32.84.191    1025   
24068   2011-08-18  10:43:18.180  0.000   UDP     147.32.80.9      53   
24069   2011-08-18  10:43:18.182  2.984   UDP   147.32.84.191     123   
29409 

In [38]:
# the histogram of the data
categories = ["Date", "Start", "Dur", "Prot", "Src", "PortSrc", "Dest", 
                "PortDest", "Flags", "Tos", "Packets", "Bytes", "Flows", "Label"]
infected_host = "147.32.84.191"
non_infected = "147.32.96.69"
infected_data = data.loc[(data["Src"] == infected_host) | data["Dest"].isin([infected_host]), :]
non_infected_data = data.loc[(data["Src"] == non_infected) | data["Dest"].isin([non_infected]), :]
for cat in categories:
    print non_infected_data[cat].value_counts()


2011-08-18    316219
Name: Date, dtype: int64
14:47:08.199    7
12:32:36.346    7
12:27:34.650    7
12:22:10.707    7
12:27:44.921    7
12:30:47.003    7
14:43:54.352    7
12:22:39.515    7
12:19:53.462    7
12:32:46.920    7
12:27:34.660    7
12:28:34.409    7
12:29:30.259    7
12:32:41.675    7
12:25:19.916    6
14:48:19.287    6
14:46:53.278    6
12:27:13.531    6
12:21:26.279    6
12:27:03.903    6
12:20:41.727    6
12:23:36.540    6
12:20:25.894    6
12:21:23.685    6
14:46:03.012    6
12:18:43.280    6
12:27:00.816    6
12:26:05.828    6
12:21:04.546    6
12:19:24.201    6
               ..
12:33:42.743    1
12:23:26.516    1
15:04:48.067    1
12:26:37.390    1
12:34:19.607    1
12:22:25.650    1
12:29:53.400    1
12:20:17.427    1
14:49:11.679    1
12:20:17.424    1
14:50:21.543    1
14:49:11.676    1
12:10:04.976    1
12:27:02.588    1
12:32:39.708    1
12:34:20.866    1
12:28:22.262    1
12:32:39.702    1
12:27:02.582    1
12:27:02.583    1
12:27:02.585    1
12:22:28.554    1


2011-08-18    645358
Name: Date, dtype: int64
